In [237]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
df=pd.read_csv("detailed_commits_all_stars_.csv")

In [233]:
df

,title,year,authors,author_len,H5Index,ImpactScore,conf_date,citations,key,subscribers,...,detailed_total_month,detailed_stars_day,detailed_stars_week,detailed_stars_month,contributors_count,contributors_variance,languages_length,languages,top_lang,language_variance
0,Transfer Learning by Kernel Meta-Learning.,2012,Fabio Aiolli,1,171,18.48,2011-06-28 00:00:00,11.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bandits with Generalized Linear Models.,2012,Lihong Li 0001/Wei Chu/John Langford 0001/Taes...,5,171,18.48,2011-06-28 00:00:00,49.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Winning The Transfer Learning Track of Yahoo!'...,2011,Andrey Gulin/Igor Kuralenok/Dmitry Pavlov,3,171,18.48,2010-06-21 00:00:00,44.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,The Yahoo! Music Dataset and KDD-Cup '11.,2012,Gideon Dror/Noam Koenigstein/Yehuda Koren/Mark...,4,90,13.53,2011-08-21 00:00:00,274.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Rating Prediction with Informative Ensemble of...,2012,Zhao Zheng/Tianqi Chen/Nathan Nan Liu/Qiang Ya...,5,90,13.53,2011-08-21 00:00:00,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37601,Deceptive Answer Prediction with User Preferen...,2013,Fangtao Li/Yang Gao/Shuchang Zhou/Xiance Si/De...,5,135,10.56,2013-08-04 00:00:00,448.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37602,Fine-grained Semantic Typing of Emerging Entit...,2013,Ndapandula Nakashole/Tomasz Tylenda/Gerhard We...,3,135,10.56,2013-08-04 00:00:00,111.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37603,Lemmatisation as a Tagging Task.,2012,Andrea Gesmundo/Tanja Samardzic,2,135,10.56,2012-07-08 00:00:00,50.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37604,Co-Regression for Cross-Language Review Rating...,2013,Xiaojun Wan 0001,1,135,10.56,2013-08-04 00:00:00,8.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df["commits"]=df["commit_after_publication"]+df["commmit_before_publication"]
df["popularity"]=df["stars"]+df["forks"]
df=df[~(df["year"]==2020)]
df["citations"].fillna(0,inplace=True)
df[((df["stars"]==0) & (df["detailed_stars_week"]=="[]"))]["detailed_stars_week"]="[(0,0)]"
df.sort_values(by="popularity",ascending=True,inplace=True)
df.drop_duplicates("title",inplace=True)

In [235]:
df=df[~df['detailed_stars_week'].isna()]
df=df[~(df['detailed_stars_week']=="[]")]

In [ ]:
df

In [216]:
df["id"]=df["key"].str.split("/",1).str[1]

In [217]:
import requests
def get_readme_request(repo_id):
    format_string=f"https://raw.githubusercontent.com/{repo_id}/master/README.md"
    req=requests.get(format_string)
    
    if req.status_code==200:
        return req.text
    else:
        return req.status_code

In [ ]:
readmes=[]
for val in df["id"]:
    readmes.append(get_readme_request(val))

In [177]:
df['Readmes']=readmes

In [178]:
df["Readmes"]=df["Readmes"].astype(str)

In [179]:
df.to_csv("detailed_commits_all_wreadmes.csv",index=False)

In [239]:
df=pd.read_csv("detailed_commits_all_wreadmes.csv")

In [ ]:
df

In [244]:
df["Readmes"]=df["Readmes"].astype(str)

In [ ]:
df.head(10)

In [245]:
import re
zero_list=[]
pub_link=[]
for row in df.itertuples():
    #removes the full stop
    if row.title.lower()[:-1] in str(row.Readmes).lower():
        zero_list.append(1)
    else:
        loop_bol=False
        for name in row.authors.split("/"):
            if name.lower() in row.Readmes.lower():
                loop_bol=True
                zero_list.append(1)
                break
                
        if not loop_bol:
            zero_list.append(0)
                
        
    if ("doi" in row.Readmes.lower()) or ("arxiv" in row.Readmes.lower()) or ("cite" in row.Readmes.lower()) or ("@article" in row.Readmes.lower()) or ("@inproceedings" in row.Readmes.lower()) or ("@misc" in row.Readmes.lower()) :
        pub_link.append(1)
    else:
        pub_link.append(0)

In [246]:
df['Matched']=zero_list
df["Academic"]=pub_link

In [ ]:
for row in df.itertuples():
    if row.Matched!=1 and row.Academic==1:
        print(row.title,row.key,row.Matched,row.Academic)

In [227]:
df.groupby(["Matched", "Academic"]).size().unstack(fill_value=0)

Academic,0,1
Matched,,
0,953,509
1,443,1635


In [ ]:
df.groupby(["Matched", "Academic"]).size().unstack(fill_value=0)

In [229]:
df["Matched"].value_counts()

1    2078
0    1462
Name: Matched, dtype: int64

In [230]:
df["Academic"].value_counts()

1    2144
0    1396
Name: Academic, dtype: int64

In [253]:
sample=df.sample(347)

In [256]:
sample.columns

Index(['title', 'year', 'authors', 'author_len', 'H5Index', 'ImpactScore',
       'conf_date', 'citations', 'key', 'subscribers', 'stars', 'forks',
       'size', 'OpenIssues', 'activity_range', 'commit_rate',
       'commmit_before_publication', 'commit_after_publication',
       'commits_weekly', 'detailed_commit_day', 'detailed_commit_week',
       'detailed_commit_month', 'detailed_total_day', 'detailed_total_week',
       'detailed_total_month', 'detailed_stars_day', 'detailed_stars_week',
       'detailed_stars_month', 'contributors_count', 'contributors_variance',
       'languages_length', 'languages', 'top_lang', 'language_variance',
       'commits', 'popularity', 'id', 'Readmes', 'Matched', 'Academic'],
      dtype='object')

In [254]:
sample.groupby(["Matched","Academic"]).size().unstack(fill_value=0)

Academic,0,1
Matched,,
0,80,41
1,48,178


In [257]:
sample[["title", 'year', 'authors','citations', 'key','Matched', 'Academic']].to_csv("tested_sample.csv")